# Data Section

##### Data of this project is mainly the coordinates of Halifax. 
##### The first step is to find out the communities, area or neighborhood classification of Halifax. The first idea is to search in Wikipedia. Hence, the communities is found in https://en.wikipedia.org/wiki/Communities_in_the_Halifax_Regional_Municipality. There are four urban communities was captured into the study: Halifax Panasula, Mainland Halifax, Dartmouth and Bedford. The sub-communities was scrapied by using the BeautifulSoup package. Also, the coordinates of each communities was captured from Wikipedia link.
##### The second step would be passing the coordinates to Foursquare database and explore the venues of the communities and transform into map and find the areas with low supply of restaurants. 
##### However, during the programming section, the coordinates from Wikipedia is not competable for folium package. Therefore, geocoder package was used for obtain the coordinates for Halifax commmunities. This step is to pass the communities' names to geocoder and generate coordinates for mapping. 

In [1]:
#Install packages
#! pip install beautifulsoup4
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/zhenglouis/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2019.11.27-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                          anaconda --> conda-forge
  conda                                            anaconda --> conda-forge
  openssl                anaconda::open

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import json 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt  
from matplotlib import style 
print("done")

done


In [7]:
#Search halifax communities in wikipedia, and extract them from Halifax Peninsula.
HalifaxUrl = 'https://en.wikipedia.org/wiki/Communities_in_the_Halifax_Regional_Municipality'
WholeHalifax = BeautifulSoup(requests.get(HalifaxUrl).text, 'lxml')
Halifax_Peninsula = WholeHalifax.find('table', class_="multicol").tbody.tr
HalifaxCommunities = []
for i in Halifax_Peninsula.find_all('li'):
    HalifaxCommunities.append(i.text)
for n, i in enumerate(HalifaxCommunities):
    if i == 'Albro Lake\nCrystal Heights\nHighfield Park':
        HalifaxCommunities[n] = 'Albro Lake'
    if i == 'Burnside':
        HalifaxCommunities[n] = 'Burnside, Halifax'
    if i == 'Cowie Hill':
        HalifaxCommunities[n] = 'Cowie Hill, Halifax'
    if i == 'Woodlawn':
        HalifaxCommunities[n] = 'Woodlawn, Halifax'
    if i == 'Manor Park':
        HalifaxCommunities[n] = 'Manor Park, Halifax'
    if i == 'Wellington':
        HalifaxCommunities[n] = 'Wellington, Halifax'
    if i == 'Waverley':
        HalifaxCommunities[n] = 'Waverley, Halifax'
    if i == 'Quinpool district':
        HalifaxCommunities[n] = 'Quinpool, Halifax'
    if i == 'Fairview':
        HalifaxCommunities[n] = 'Fairview, Halifax'
    if i == 'Fairmount':
        HalifaxCommunities[n] = 'Fairmount, Halifax'
    if i == 'Woodside':
        HalifaxCommunities[n] = 'Woodside, Halifax'
    if i == 'Richmond':
        del HalifaxCommunities[n]

In [8]:
#Append HalifaxCommunities list mannually and obtain communities coordinates from Geolocator.
lat = []
lon = []
for i in HalifaxCommunities:
    address = i + ', CA'
    geolocator = Nominatim(user_agent = "YHZ_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lat.append(latitude)
    lon.append(longitude)
    print(i,location.latitude,location.longitude)
YHZcor = pd.DataFrame({'communities':HalifaxCommunities, 'lat':lat, 'lon':lon})
YHZcor.head()

Downtown Halifax 44.6461752 -63.5725221
The Hydrostone 44.6641477 -63.6037432856765
North End Halifax 44.6550339 -63.5949305
West End, Halifax 44.6481851 -63.6172772
Quinpool, Halifax 44.64717195 -63.5965004821299
South End Halifax 44.6318333 -63.5800267
Spring Garden 44.64128565 -63.5811858125625
Armdale 44.6406015 -63.6224485
Boulderwood 44.6245031 -63.5982442
Clayton Park 44.6639666 -63.6525669
Cowie Hill, Halifax 44.6285559 -63.6215005
Fairview, Halifax 44.65515 -63.6384566
Fairmount, Halifax 44.6424475 -63.6324446
Jollimore 44.6299041 -63.6063922
Melville Cove 44.6330952 -63.6153365
Rockingham 44.6760232 -63.6616731
Spryfield 44.6174989 -63.6183631
Albro Lake 44.6853126 -63.5749927
Crystal Heights 44.6874216 -63.5833075
Highfield Park 44.6869672 -63.588839
Brightwood 44.6761567 -63.5772693
Burnside, Halifax 44.712138 -63.5902581
Crichton Park 44.6810198 -63.5657037
Downtown Dartmouth 44.6675083 -63.5686004
Imperoyal 44.6372717 -63.5253527
Manor Park, Halifax 44.6706939 -63.5459733

,communities,lat,lon
0,Downtown Halifax,44.646175,-63.572522
1,The Hydrostone,44.664148,-63.603743
2,North End Halifax,44.655034,-63.594930
3,"West End, Halifax",44.648185,-63.617277
4,"Quinpool, Halifax",44.647172,-63.596500


In [9]:
#Obtain the geograpical coordinate of Halifax
address = 'Halifax, CA'
geolocator = Nominatim(user_agent="YHZ_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat.append(latitude)
lon.append(longitude)
print('The geograpical coordinates of Halifax are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Halifax are 44.648618, -63.5859487.


In [10]:
#Maping Toronto's neighbourhood
YHZmap = folium.Map(location=[latitude, longitude], zoom_start=10, width = 800, height = 1000)

for lat, lng, borough in zip(YHZcor['lat'], YHZcor['lon'], YHZcor['communities']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(YHZmap)  
    
YHZmap

In [12]:
#Exploring the venues of Halifax
CLIENT_ID = '53IPUQGU1JWK24WFB1GCTYYXQ5MP4EQRCHRIDKTHLCSREIRP' 
CLIENT_SECRET = '4UCVZQWCTSAH3NW1XFKTI4MVEOLHJY1AT4LGDANRC2XP4LHN' 
VERSION = '20180605'
limit = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Halifax_venues = getNearbyVenues(names=YHZcor['communities'],
                                   latitudes=YHZcor['lat'],
                                   longitudes=YHZcor['lon']
                                  )

In [13]:
Halifax_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Halifax,44.646175,-63.572522,Stillwell Beer Bar,44.646908,-63.573875,Bar
1,Downtown Halifax,44.646175,-63.572522,Waterfront Boardwalk,44.647040,-63.570178,Waterfront
2,Downtown Halifax,44.646175,-63.572522,The Old Triangle Irish Alehouse,44.647654,-63.572307,Pub
3,Downtown Halifax,44.646175,-63.572522,The Stubborn Goat Gastropub,44.645389,-63.574467,Gastropub
4,Downtown Halifax,44.646175,-63.572522,Chives Canadian Bistro,44.645078,-63.572802,Restaurant
...,...,...,...,...,...,...,...
449,"Waverley, Halifax",44.782712,-63.598929,Open Door Construction,44.779710,-63.596966,Construction & Landscaping
450,"Waverley, Halifax",44.782712,-63.598929,Wicky Clarke's Country Club,44.786018,-63.599890,Playground
451,"Wellington, Halifax",44.872754,-63.607648,Inner Strength Taekwondo,44.869598,-63.610799,Athletics & Sports
452,Windsor Junction,44.790638,-63.646073,Bobby Grace Furniture,44.791338,-63.643112,Furniture / Home Store
